This file is just here to play and prepare the test, do not care about the content

In [ ]:
from analink.parser.base import Container

In [ ]:
a=Container(content="a")

In [ ]:
a

In [ ]:
a

In [ ]:
del(a)

In [ ]:
Container(content='a')

In [ ]:
Container(content='b')

In [ ]:
from analink.models import Choice, ChoiceType
from analink.parser.knots import parse_knots

In [ ]:
ink_code = """
"What's that?" my master asked.
*	"I am somewhat tired[."]," I repeated.
	"Really," he responded. "How deleterious."
*	"Nothing, Monsieur!"[] I replied.
	"Very good, then."
*  "I said, this journey is appalling[."] and I want no more of it."
	"Ah," he replied, not unkindly. "I see you are feeling frustrated. Tomorrow, things will improve."
"""

In [ ]:
parse_knots(ink_code)

In [ ]:
def count_leading_chars(line: str, char: str) -> int:
    """Count leading characters (for nesting level)"""
    count = 0
    for c in line:
        if c == char:
            count += 1
        elif c == " " or c == "\t":
            continue
        else:
            break
    return count

In [ ]:
lines = ink_code.split("\n")

In [ ]:
def parse_line(line: str):
    # check how many leading character
    only_once_choice = count_leading_chars(line, "*")
    sticky_choice = count_leading_chars(line, "+")
    weave = count_leading_chars(line, "-")
    knot_stitches = count_leading_chars(line, "=")
    diverts = count_leading_chars(line, "->")
    return only_once_choice, sticky_choice, weave, knot_stitches, diverts

In [ ]:
import re

In [ ]:
def find_bracket_patterns(text):
    # Pattern explanation:
    # (?<!\\) - negative lookbehind: not preceded by backslash
    # \[ - literal opening bracket
    # ([^\\]*(?:\\.[^\\]*)*) - content that handles escaped characters properly
    # \] - literal closing bracket
    # (?<!\\) - negative lookbehind: closing bracket not preceded by backslash

    pattern = r"(?<!\\)\[([^\\]*(?:\\.[^\\]*)*)\](?<!\\)"

    matches = re.findall(pattern, text)
    return matches




In [ ]:
lines_list = lines
i = 0
choices=[]
while i < len(lines_list):
    line = lines_list[i]
    stripped = line.strip()
    if count_leading_chars(stripped, "*") > 0:
        line_number = i
        choice_buffer = [line.lstrip("*").strip()]
        i += 1
        while i < len(lines_list):
            line = lines_list[i]
            stripped = line.strip()
            only_once_choice, sticky_choice, weave, knot_stitches, diverts = parse_line(
                stripped
            )
            if count_leading_chars(stripped, "*") == 0:
                choice_buffer.append(stripped)
                i += 1
            else:
                # Don't increment i here - we want to process this line in the outer loop
                break
        # print(choice_buffer)
        choices.append(parse_only_once_choice(choice_buffer, line_number))
    else:
        i += 1

In [ ]:
choices

In [ ]:
lines_generator = enumerate(lines)

In [ ]:
lines_list = lines
i = 0
while i < len(lines_list):
    line = lines_list[i]
    stripped = line.strip()
    only_once_choice, sticky_choice, weave, knot_stitches, diverts = parse_line(
        stripped
    )

    if knot_stitches == 1:
        # TODO : parse_stitches(stripped?)
        i += 1
        continue
    elif knot_stitches >= 2:
        # TODO : parse_knot(?)
        i += 1
        continue
    elif weave > 0:
        # TODO : parse_weave(?)
        i += 1
        continue
    elif sticky_choice > 0:
        # TODO : parse_sticky_choice(?)
        i += 1
        continue
    elif only_once_choice > 0:
        choice_buffer = line
        i += 1
        while i < len(lines_list):
            line = lines_list[i]
            stripped = line.strip()
            only_once_choice, sticky_choice, weave, knot_stitches, diverts = parse_line(
                stripped
            )
            if only_once_choice == 0:
                choice_buffer += "\n" + line
                i += 1
            else:
                # Don't increment i here - we want to process this line in the outer loop
                break
        parse_only_once_choice(choice_buffer)
        print(choice_buffer)
        continue
    elif diverts > 0:
        # TODO : parse_diverts(?)
        i += 1
        continue
    else:
        i += 1
        continue